# Tech Team Ticket Metrics

## Retrieve Current Data

In [ ]:
# SPDX-License-Identifier: Apache-2.0
# Licensed to the Ed-Fi Alliance under one or more agreements.
# The Ed-Fi Alliance licenses this file to you under the Apache License, Version 2.0.
# See the LICENSE and NOTICES files in the project root for more information.

from edfi_tech_metrics.settings import load_from_env, Configuration
from edfi_tech_metrics.jira import JiraBrowser
from edfi_tech_metrics.ticket_age import write_ticket_age_files, generate_ticket_age_plots

conf: Configuration = load_from_env()
#conf.log_level = "DEBUG"
browser = JiraBrowser(conf)

portfolios = {
    "ODS Platform": ["ODS", "PERF"],
    "Team B": ["METAED", "DMS", "ADMINAPI"],
    "Ed-Fi Tools": ["AA", "AC", "APIPUB", "DI", "EPPETA"],
    "Data Standard": [ "DATASTD", "MODL", "TPDMDEV"],
    "Support": ["EDFI"]
}

projects = [i for _, v in portfolios.items() for i in v]
df = browser.get_unresolved_tickets(projects)

In [ ]:
write_ticket_age_files(conf, df)

## Ticket Age

The plots below are [boxplots](https://www.atlassian.com/data/charts/box-plot-complete-guide). They middle line
is the median value and the box dimensions are sized to the first and last quartiles. In other words, 25% of all
the records are to the left of the box, 50% are inside the box, and 25% are to the right. These boxplots are
good for looking at the change in a distribution over time. The value printed above the box is the mean. The
mean will typically be close to the median, but not exactly the same. The mean is our primary statistic, but
the quartiles in the boxplot are also interesting: we might see only a small change in the mean, but a big
change in the median. That is usually a good sign.

If a plot only has one box in it, that means it was the first month in which we reported on that project.

### All Tickets

In [ ]:
generate_ticket_age_plots("./data/ticket-age", projects)

### Filtered Tickets

These plots exclude "Test" type tickets and those that have an assigned Fix Version.

In [ ]:
generate_ticket_age_plots("./data/ticket-age-filtered", projects)

## Backlog Health

Enter current team average velocity over the last five sprints below:

In [ ]:
from edfi_tech_metrics.backlog import run_backlog_health_ui
run_backlog_health_ui(df, portfolios)